<a href="https://colab.research.google.com/github/Anupama83/PyTorch-for-Deep-Learning/blob/main/Downloading_video_files_from_YouTube_Converting_to_Audio_Remove_VAD_Chunk_NLTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# To download video files from YouTube
## 1. Install and import important libraries


In [2]:
!apt-get update
!apt-get install -y xvfb

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.1 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

In [3]:
!pip install --upgrade pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.7 MB/s eta 0:00:00


In [4]:
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.2 MB/s eta 0:00:00


In [ ]:
import yt_dlp

def download_with_ytdlp(url, output_path):
    ydl_opts = {
        'format': 'bestvideo+bestaudio',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'merge_output_format': 'mp4',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print("Download completed.")

# Pass url of youtube video
url = "https://www.youtube.com/shorts/A3v9GrVxnNg"
# mention the path where you want to store your downloaded video file
output_path = "/content/drive/MyDrive/Research Work_Zero_Shot_Learning/Video Files for Dataset/Nirmala Sitaraman"
download_with_ytdlp(url, output_path)


# Converting video file to audio file

In [15]:
from moviepy.editor import AudioFileClip
import os

def convert_mp4_to_wav(mp4_path, wav_path=None):
    # Set the output path to the same directory with .wav extension if not provided
    if wav_path is None:
        wav_path = os.path.splitext(mp4_path)[0] + '.wav'

    # Load the audio from the .mp4 file
    with AudioFileClip(mp4_path) as audio_clip:
        # Write the audio to .wav file
        audio_clip.write_audiofile(wav_path, codec='pcm_s16le')

    print(f"Conversion complete. Audio saved as: {wav_path}")

# Pass the path of downloaded video file
mp4_path = "/content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/Venkaiah Naidu Last Speech In Parliament As Vice President I Was In Tears Didnt Ask For This.mp4"  # Replace with the path to your .mp4 file
convert_mp4_to_wav(mp4_path)


MoviePy - Writing audio in /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/Venkaiah Naidu Last Speech In Parliament As Vice President I Was In Tears Didnt Ask For This.wav


MoviePy - Done.
Conversion complete. Audio saved as: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/Venkaiah Naidu Last Speech In Parliament As Vice President I Was In Tears Didnt Ask For This.wav


# VAD, Noise removing and Chunk the audio files

In [8]:
#@title Install and Import Dependencies

# this assumes that you have a relevant version of PyTorch installed
!pip install -q torchaudio

SAMPLING_RATE = 16000

import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint
# download example
#torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')

In [9]:
USE_PIP = True # download model using pip package or torch.hub
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime
if USE_PIP:
  !pip install -q silero-vad
  from silero_vad import (load_silero_vad,
                          read_audio,
                          get_speech_timestamps,
                          save_audio,
                          VADIterator,
                          collect_chunks)
  model = load_silero_vad(onnx=USE_ONNX)
else:
  model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                model='silero_vad',
                                force_reload=True,
                                onnx=USE_ONNX)

  (get_speech_timestamps,
  save_audio,
  read_audio,
  VADIterator,
  collect_chunks) = utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00


In [16]:
import os
import torchaudio
from pprint import pprint

# Function to save speech chunks to a specified folder
def save_audio_chunk(wav, start_time, end_time, output_folder, chunk_index):
    # Convert start_time and end_time to sample indices
    # Added error handling for invalid start_time and end_time
    try:
        start_sample = int(float(start_time) * SAMPLING_RATE)  # Convert to float first
        end_sample = int(float(end_time) * SAMPLING_RATE)  # Convert to float first
    except ValueError:
        print(f"Skipping chunk {chunk_index} due to invalid start_time or end_time: start_time={start_time}, end_time={end_time}")
        return  # Skip this chunk

    # Extract the chunk
    chunk = wav[:, start_sample:end_sample]

    # Define the output path for the chunk
    output_path = os.path.join(output_folder, f"chunk_{chunk_index}.wav")

    # Save the chunk as a WAV file
    torchaudio.save(output_path, chunk, SAMPLING_RATE)
    print(f"Saved chunk {chunk_index} to {output_path}")

# Function to process the audio, chunk it, and save the chunks
def process_audio_and_save_chunks(input_audio_path, output_folder, model, sampling_rate=16000):
    # Load the full audio file
    wav, _ = torchaudio.load(input_audio_path, normalize=True)

    # Check if audio has more than one channel and convert to mono if necessary
    if wav.shape[0] > 1:
        wav = wav.mean(dim=0, keepdim=True)  # Convert to mono by averaging channels

    # Get speech timestamps (assumes get_speech_timestamps is defined and model is loaded)
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sampling_rate)
    pprint(speech_timestamps)  # Optionally print the timestamps

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over the speech timestamps and save each chunk
    for chunk_index, (start_time, end_time) in enumerate(speech_timestamps):
        save_audio_chunk(wav, start_time, end_time, output_folder, chunk_index)


# Example usage
input_audio_path = '/content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/Venkaiah Naidu Last Speech In Parliament As Vice President I Was In Tears Didnt Ask For This.wav'
output_folder = '/content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/'
SAMPLING_RATE = 16000  # Define your sampling rate
# Make sure to load your model here
model = load_silero_vad()  # Assuming you are using silero-vad


# Call the process function
process_audio_and_save_chunks(input_audio_path, output_folder, model, sampling_rate=SAMPLING_RATE)

[{'end': 57824, 'start': 4640}, {'end': 66016, 'start': 59936}]
Skipping chunk 0 due to invalid start_time or end_time: start_time=start, end_time=end
Skipping chunk 1 due to invalid start_time or end_time: start_time=start, end_time=end


In [12]:
!pip install pydub

In [18]:
from pydub import AudioSegment
import os

def split_wav_file(input_file, output_dir, chunk_duration=20):
    """
    Splits a .wav file into chunks of a specified duration.

    :param input_file: Path to the input .wav file.
    :param output_dir: Directory to save the chunks.
    :param chunk_duration: Duration of each chunk in seconds (default is 20 seconds).
    """
    try:
        # Load the .wav file
        audio = AudioSegment.from_wav(input_file)
        total_duration = len(audio)  # Total duration in milliseconds

        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Split the audio into chunks
        chunk_duration_ms = chunk_duration * 1000  # Convert to milliseconds
        for i in range(0, total_duration, chunk_duration_ms):
            chunk = audio[i:i + chunk_duration_ms]
            chunk_name = f"chunk_{i // chunk_duration_ms + 1}.wav"
            chunk_path = os.path.join(output_dir, chunk_name)
            chunk.export(chunk_path, format="wav")
            print(f"Exported: {chunk_path}")

        print("Splitting complete.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_wav = "/content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/Venkaiah Naidu Last Speech In Parliament As Vice President I Was In Tears Didnt Ask For This.wav"
output_directory = "/content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar" # Create a folder with speaker name and pass that path here
split_wav_file(input_wav, output_directory)


Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_1.wav
Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_2.wav
Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_3.wav
Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_4.wav
Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_5.wav
Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_6.wav
Exported: /content/drive/MyDrive/Research Work_Speaker Recognition/Dataset videos/chunked files_Jagdeep_Dhankar/Jagdeep Dhankar/chunk_7.wav
Exported: /content/d